# 使用新方法计算F1
- F1计算方式为表格中的信息被模型抽取到

In [31]:
#file_path = 'results/enhance-rare-test-llama-output.json'
file_path = 'improve_data/提示词修改后结果.jsonl'
# file_path = 'results/no-rare-test-demo-result.json'
# file_path = 'results/baichuan7B-ft-test-output-v2.json'
# file_path = 'results/baichuan2-13B-iepile-ft-test-output.json'

In [73]:
import json

excluded_entity_types = {"任务", "事件", "行动"}
# excluded_entity_types = {'任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '行动', '军种', '数量', '任务状态', '开始时间', '装备', '单位', '人数', '国别',}


def count_entities_in_jsonl(file_path):
    len_label = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line)

            # 获取 label 字段并解析为列表
            label = json.loads(record['label'])

            for la in label:
                if la['entity_type'] in excluded_entity_types:
                    continue
                len_label += 1
                
    # “6”表示headers总数22（25种除去行动事件任务）/split_num（一次抽取4个实体类型）后向上取整
    return len_label / 6


# 设置你的文件路径
total_entities = count_entities_in_jsonl(file_path)
print(f"总实体数量: {total_entities}")


总实体数量: 1420.0


In [74]:
import json

excluded_entity_types = {"任务", "事件", "行动"}
# excluded_entity_types = {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '行动', '军种', '数量', '任务状态', '时间', '地点', '人物', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '事件', '国家', '任务', '国别', '装备数量'}

def count_valid_entities(file_path):
    valid_entity_count = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line)

            # 解析 label 和 output 字段
            label_entities = json.loads(record['label'])
            output_entities = json.loads(record['output'])

            # 将 output 中的实体合并成一个列表
            output_entity_list = []
            for key in output_entities:

                output_entity_list.extend(output_entities[key])

            # print(output_entity_list)

            # 检查 label 中的每个实体是否在 output 中逐字匹配
            for entity_info in label_entities:
                label_entity = entity_info['entity']
                entity_type = entity_info['entity_type']

                if entity_type in excluded_entity_types:
                    continue

                # 检查是否 label_entity 在 output_entity_list 中的任何一个实体字符串中
                if any(label_entity in output_entity for output_entity in output_entity_list):
                    valid_entity_count += 1
                # else:
                #     print(label_entity)

    return valid_entity_count


# 设置你的文件路径
valid_entity_count = count_valid_entities(file_path)
print(f"有效实体数量: {valid_entity_count}")

# 
print(f"F1值：{valid_entity_count / total_entities}")

有效实体数量: 1239
F1值：0.8725352112676056


{
  '组织机构':0.769
  '任务类型': 
  '备注':
  '指挥员': 
  '主体数量':0.819 
  '主体名称':0.851
  '主体类型':0.811
  '军种': 
  '数量': 
  '任务状态': 
  '时间': 0.901
  '地点': 0.913
  '人物': 0.08 # 原始数据标注质量不好
  '开始时间': 
  '装备名称':0.866 
  '装备':
  '单位': 
  '人数': 
  '装备类型': 0.841
  '国家': 0.365 
  '国别':
  '装备数量':0.817
}

## 计算F1

In [35]:
import json

excluded_entity_types = {"时间"}
#excluded_entity_types = {"任务-时间", "任务-主体名称", "主体名称-主体类型", "主体名称-主体数量", "任务-组织机构", "任务-地点", "任务-军种", "任务-单位", "任务-人数", "任务-指挥员", "任务-任务状态", "行动-时间", "行动-主体名称", "行动-组织机构", "行动-地点", "行动-军种", "行动-单位", "行动-人数", "行动-指挥员", "行动-任务状态", "事件-时间", "事件-主体名称", "事件-组织机构", "事件-地点", "任务-国家", "任务-人物", "行动-国家", "行动-人物", "行动-国家", "行动-人物"}

# excluded_entity_types = {"核心事件-时间","核心事件-主体名称","主体名称-主体类型","主体名称-主体数量","核心事件-组织机构","核心事件-地点","核心事件-军种", "核心事件-单位", "核心事件-人数", "核心事件-指挥员", "核心事件-任务状态", "核心事件-国家","核心事件-人物"}

# excluded_entity_types = {"时间", "相关主体", "类型", "组织机构", "地点", "军种", "单位", "任务状态"}
# excluded_entity_types = {"时间", "相关主体", "类型", "组织机构", "地点"}
# excluded_entity_types = {'组织机构', '任务类型', '备注', '指挥员', '主体数量', '主体名称', '主体类型', '军种', '数量', '任务状态', '时间', '地点', '开始时间', '装备名称', '装备', '单位', '人数', '装备类型', '国别', '装备数量'}


def count_valid_relations(file_path):
    valid_tail_count = 0
    relation_count = 0

    # 打开 JSONL 文件
    with open(file_path, 'r', encoding='utf-8') as file:
        # # 逐行读取文件内容
        for line in file:
            # 解析 JSON 数据
            record = json.loads(line.strip())
            #record = json.load(line)
            # print(record)
            
            # demo测试自己准备数据用这个
            # 解析 label 和 output 字段
            label_relations = record['relation']
            # print(label_relations)
            output_relations = json.loads(record['output'])
            #print(output_relations)

            # #脚本整体输出用这个
            # # 解析 label 和 output 字段
            # label_relations = json.loads(record['label'])
            # # print(label_relations)
            # output_relations = json.loads(record['output'])
            # #print(output_relations)
            # 将 output 中的关系合并成一个列表
            output_relation_list = []
            for key, value in output_relations.items():
                #output_relation_list.extend(output_relations[key])
                output_relation_list.append(key)

            # print(output_relation_list)

            # 检查 label 中的每个实体是否在 output 中逐字匹配
            # 对 label数组 中的每一项关系对象拿到 头尾实体 和 关系类型
            for relation_info in label_relations:
                label_head = relation_info['head']
                label_relation_type = relation_info['relation']
                label_tail = relation_info['tail']

                # 如果 label 中的关系类型在候选关系类型集合中
                if label_relation_type in excluded_entity_types:
                    relation_count += 1
                    # 获取label中当前关系的尾实体类型
                    label_tail_type = label_relation_type
                    for key, value in output_relations.items():
                        result_tail_type = key
                        # 如果当前 label中 的 relation类型 对应到了 output中的 relation类型
                        if result_tail_type == label_tail_type:
                            # 进一步比对抽取出的尾实体是不是一致
                            # 因为头实体是行动任务啥的具有不确定性，所以只对比尾实体
                            result_triple_head = []
                            result_triple_tail = []
                            for triple in value:
                                # result_triple_head.append(triple["head"])
                                result_triple_tail.append(triple["tail"])
                                # print(triple["tail"])
                            for item in result_triple_tail:
                                # 这里是判断抽取成功的条件，现在是简单的字符串判断后续可以更换比对函数
                                if label_tail in item or item in label_tail:
                            # if label_tail in result_triple_tail:
                            # if any(label_tail in result_triple_tail for output_relation in output_relation_list):
                                    valid_tail_count += 1
                                   # print(label_tail)

    return valid_tail_count, relation_count / 1


# 设置你的文件路径
valid_tail_count, relations = count_valid_relations(file_path)
print(f"有效关系数量: {valid_tail_count}，总关系数量：{relations}")

# 算F1
print(f"F1值：{valid_tail_count / relations}")

有效关系数量: 59，总关系数量：91.0
F1值：0.6483516483516484


## 统计
{
  "任务-时间":0.231
  "任务-主体名称": 测试集中没有分到这条关系
  "主体名称-主体类型":0.47
  "主体名称-主体数量"：0.428
  "任务-组织机构": 0.574
  "任务-地点":0.576 
  "任务-军种":0.333
  "任务-单位":1.0（4/4）
  "任务-人数": 0（0/1）
  "任务-指挥员": 测试集中没有分到这条关系（原始数据中指挥员内容也基本都是缺失的）
  "任务-任务状态": 0.5
  "行动-时间": 0.421
  "行动-主体名称": 测试集中没有这样的关系
  "行动-组织机构": 0.324
  "行动-地点": 0.387
  "行动-军种": 0（0/1）
  "行动-单位": 测试集中没有这样的关系
  "行动-人数": 同上
  "行动-指挥员"：同上
  "行动-任务状态": 0（0/1）
  "事件-时间": 测试集中没有这样的关系
  "事件-主体名称": 同上 
  "事件-组织机构": 同上
  "事件-地点": 同上
}

## 添加Verify字段

In [ ]:
import json
import argparse
from fuzzywuzzy import fuzz


check_header = ["任务状态", "备注", "任务", "行动", "事件"]


# 将OneKE模型的结果直接添加到input中，并生成新的output（多了verify字段，verify来自匹配成功的result）
def add_verify_data(input_data, result_data):
    data = input_data
    result_data = result_data  # 直接使用传入的result_data

    # 遍历每个元素
    for item in data:
        headers = item['table']['header']
        table_data = item['table']['data']
        verify = {}

        # 初始化verify字典的每一行
        for row_key in table_data.keys():
            verify[row_key] = []

        # 对每个header进行处理
        for index, header in enumerate(headers):
            # 获取对应的result_values
            result_values = result_data.get(header, [])

            # 遍历table_data中的每一行
            for row_key, row_values in table_data.items():
                value = row_values[index]

                if value == "":
                    verify[row_key].append(None)
                    continue

                matched_results = []

                # 如果包含分号，进行拆分
                sub_values = value.split('；') if '；' in value else [value]

                # 与result_values中的值比较
                for sub_value in sub_values:
                    for result_value in result_values:
                        # 计算相似度
                        similarity = fuzz.ratio(sub_value.strip(), result_value)
                        if similarity > 80:
                            matched_results.append(result_value)

                if matched_results:
                    # 将多个匹配的result_value用分号连接起来
                    verify_value = '；'.join(matched_results)
                    verify[row_key].append(verify_value)
                else:
                    verify[row_key].append(None)

        item['verify'] = verify

    return data


# 对text分句
def split_text(text):
    # 按逗号和句号分句
    sentences = text.replace('，', ',').replace('。', '.').split(',')
    sub_sentences = []
    for sentence in sentences:
        sub_sentences.extend(sentence.split('.'))
    return [s.strip() for s in sub_sentences if s.strip()]


# 滑动窗口
def sliding_window_compare(value, sub_sentences):
    first_char = value[0]
    best_match = None
    highest_similarity = 0

    # 遍历每个子句
    for sub_sentence in sub_sentences:
        # 找到包含首字符的位置
        start_positions = [i for i in range(len(sub_sentence)) if sub_sentence[i] == first_char]

        for start in start_positions:
            # 滑动窗口进行比较
            for end in range(start + 1, len(sub_sentence) + 1):
                window = sub_sentence[start:end]
                similarity = fuzz.ratio(value, window)
                if similarity > 80 and similarity > highest_similarity:
                    highest_similarity = similarity
                    best_match = window

    return best_match


# 更新verify字段：verify来自匹配成功的data
def update_verify_based_on_text(data):
    text = data[0]['text']

    # 分句
    sub_sentences = split_text(text)

    # 遍历每个元素
    for item in data:
        table_data = item['table']['data']
        headers = item['table']['header']  # 获取header
        verify = item.get('verify', {})

        # 遍历table_data中的每一行
        for row_key, row_values in table_data.items():
            for index, value in enumerate(row_values):
                header = headers[index]  # 获取当前值对应的header

                # 判断条件：字符串长度大于等于7，或header为“任务状态”或“备注”
                if len(value) >= 7 or header in check_header:
                    best_match = sliding_window_compare(value, sub_sentences)

                    if best_match is not None:
                        # 确保verify字典中有对应的key
                        if row_key not in verify:
                            verify[row_key] = [None] * len(row_values)
                        # 将匹配的值放到verify中相应的位置
                        verify[row_key][index] = value

        # 更新item的verify字段
        item['verify'] = verify

    return data
